In [1]:
import pandas as pd
import numpy as np
# from dask import dataframe as dd
# from icd9cms.icd9 import search
import re
from matplotlib import pyplot as plt

Medicare summary file with exposure and census

In [2]:
# read in using dask
# df = dd.read_csv('/nfs/nsaph_ci3/ci3_analysis/mortality_prediction/medicare_2011_2013.csv')
# med_df = pd.read_csv('/nfs/nsaph_ci3/ci3_analysis/mortality_prediction/medicare_2011_2013.csv')
# med_df = pd.read_csv('/nfs/nsaph_ci3/ci3_analysis/mortality_prediction/medicare_2011_2016.csv')

c_size = 100000
reader = pd.read_csv('medicare_2011_2016.csv', chunksize=c_size)
df_ls = []

for chunk in reader:
    # read in chunk at a time
    c = chunk.drop(columns=['Unnamed: 0','latitude', 'longitude', 'zcta', 'qid', 'tmmx',
                     'ozone', 'pm25_nn', 'pm25_ensemble'])
    
    c_sub = c.dropna(subset=['zip'])
    df_ls.append(c_sub)
        
med_df = pd.concat(df_ls)


/nfs/projects/n/nsaph_common/conda/envs/nsaph_tensorflow/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nfs/projects/n/nsaph_common/conda/envs/nsaph_tensorflow/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


why missing Nan values for census data?
- covariates merged by zip and year
- health data (medicare) merged by zip
So, it looks like there's missing info for that zipcode & year..

In [32]:
med_df3 = med_df.dropna()
# med_df2 = df.dropna().compute()
print(med_df3.shape, med_df.shape)

group by state and zip

In [35]:
def map_sex_binary(val):
    if val == 1: # male
        ans = 0
    else: # female
        ans = 1
    return ans

med_df3['sex'] = med_df3['sex'].apply(map_sex_binary)

/nfs/tools/lib/anaconda/3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
def race_binary(med_df2):
    # convert race to binary columns
    df = pd.get_dummies(med_df2['race'])
    
    df.rename(columns={0:'race_0', 1:'race_1', 2:'race_2', 3:'race_3', 4:'race_4', 5:'race_5', 6:'race_6'}, inplace=True)
    
    med_concat = pd.concat([med_df2,df],axis=1)
    
    med_concat2 = med_concat.drop(columns=['race'])
    
    return med_concat2

med_df4 = race_binary(med_df3)

In [38]:
med_df4.shape, med_df3.shape

((229150881, 27), (229150881, 21))

In [43]:
med_df4.columns

Index(['zip', 'year', 'sex', 'age', 'statecode', 'dual', 'death', 'dead',
       'poverty', 'popdensity', 'medianhousevalue', 'pct_blk',
       'medhouseholdincome', 'pct_owner_occ', 'hispanic', 'education',
       'smoke_rate', 'mean_bmi', 'rmax', 'pr', 'race_0', 'race_1', 'race_2',
       'race_3', 'race_4', 'race_5', 'race_6'],
      dtype='object')

In [44]:
col_to_agg = {'sex':'mean', 'age':'mean', 'dual':'sum', 'death':'sum',
       'dead':'mean',  
       'poverty':'mean', 'popdensity':'mean','medianhousevalue':'mean', 
       'pct_blk':'mean', 'medhouseholdincome':'mean', 'pct_owner_occ':'mean',
       'hispanic':'mean', 'education':'mean', 'smoke_rate':'mean', 
       'mean_bmi':'mean','rmax':'mean', 'pr':'mean',
       'race_0':'mean', 'race_1':'mean', 'race_2':'mean', 'race_3':'mean', 
       'race_4':'mean', 'race_5':'mean', 'race_6':'mean'}

zip_agg = med_df4.groupby(['statecode','zip','year']).aggregate(col_to_agg1)

In [45]:
zip_agg = zip_agg.reset_index()
np.unique(zip_agg.year)

array([2011, 2012, 2013, 2014, 2015, 2016])

In [49]:
# zip_agg.to_csv("medicare_aggregated_zip_2011_2012.csv")
zip_agg.to_csv("medicare_aggregated_zip_2011_2016.csv")

Medicare Hospitalization Admission Data

In [2]:
hosp_df = pd.read_csv('/nfs/nsaph_ci3/ci3_analysis/mortality_prediction/medicare_hosp_admin_2011_2016.csv')

In [3]:
hosp_df.dropna(inplace=True)
hosp_df.reset_index(inplace=True)

First convert icd9 codes

In [5]:
def find_neoplasms(hosp_df, diag_col):
    # all fall under category neoplasms (140 - 239)
    codes = {'140-149': [], '150-159': [], '160-165': [],
        '170-176': [], '179-189':[], '190-199':[], '200-209':[],
        '210-229':[], '230-234': [], '235-238': [], '239': []}

    for i in range(hosp_df.shape[0]): #hosp_df.shape[0]
        
        # get overarching category
        c = search(hosp_df.loc[i,diag_col])
        while True:
            if c == None: # not in icd9cms dictionary
                break
            elif c.parent == None: # found overarching category of neoplasms
                break
            else: # keep searching through tree       
                c = c.parent
                
        # first check if in overarching neoplasms category (if parent is neoplasm)
        if c!= None and re.findall('140|239',str(c)):
            code = str(search(hosp_df.loc[i,diag_col]))
            if re.findall('14[0-9]',code):
                codes['140-149'].append(i)
            if re.findall('15[0-9]',code):
                codes['150-159'].append(i)
            if re.findall('16[0-5]',code):
                codes['160-165'].append(i)
            if re.findall('17[0-6]',code):
                codes['170-176'].append(i)
            if re.findall('179|18[0-9]',code):
                codes['179-189'].append(i)
            if re.findall('19[0-9]',code):
                codes['190-199'].append(i)
            if re.findall('20[0-9]',code):
                codes['200-209'].append(i)
            if re.findall('21[0-9]|22[0-9]',code):
                codes['210-229'].append(i)
            if re.findall('23[0-4]',code):
                codes['230-234'].append(i)
            if re.findall('23[5-8]',code):
                codes['235-238'].append(i)
            if re.findall('239',code):
                codes['239'].append(i)
    return codes

In [6]:
# find neoplasms in principle diagnosis
c = find_neoplasms(hosp_df, 'DIAG1')

In [7]:
# find neoplasms in secondary diagnosis
c2 = find_neoplasms(hosp_df, 'DIAG2')

In [8]:
# initialize neoplasm diagnosis binary indicators - dont run again
hosp_df['neo_140_149'] = 0
hosp_df['neo_150_159'] = 0
hosp_df['neo_160_165'] = 0
hosp_df['neo_170_176'] = 0
hosp_df['neo_179_189'] = 0
hosp_df['neo_190_199'] = 0
hosp_df['neo_200_209'] = 0
hosp_df['neo_210_229'] = 0
hosp_df['neo_230_234'] = 0
hosp_df['neo_235_238'] = 0
hosp_df['neo_239'] = 0

In [9]:
def addDiag(df,c):
    hosp_df2 = df.copy()
    
    for key in c.keys():
        r = c[key]

        if key == '140-149':
            hosp_df2.loc[r,'neo_140_149'] = 1
            
        if key == '150-159':
            hosp_df2.loc[r,'neo_150_159'] = 1
            
        if key == '160-165':
            hosp_df2.loc[r,'neo_160_165'] = 1
            
        if key == '170-176':
            hosp_df2.loc[r,'neo_170_176'] = 1
            
        if key == '179-189':
            hosp_df2.loc[r,'neo_179_189'] = 1
            
        if key == '190-199':
            hosp_df2.loc[r,'neo_190_199'] = 1
            
        if key == '200-209':
            hosp_df2.loc[r,'neo_200_209'] = 1
            
        if key == '210-229':
            hosp_df2.loc[r,'neo_210_229'] = 1
            
        if key == '230-234':
            hosp_df2.loc[r,'neo_230_234'] = 1
            
        if key == '235-238':
            hosp_df2.loc[r,'neo_235_238'] = 1
            
        if key == '239':
            hosp_df2.loc[r,'neo_239'] = 1
            
    return hosp_df2

In [10]:
# add dictionary id's together fro diag1 and diag2
c3 = {'140-149': [], '150-159': [], '160-165': [],
        '170-176': [], '179-189':[], '190-199':[], '200-209':[],
        '210-229':[], '230-234': [], '235-238': [], '239': []}

for k in c3.keys():
    c3[k] = set(c[k] + c2[k])

In [11]:
# add binary indicator of principle or secondary diagnosis code
hosp_df_diag = addDiag(hosp_df, c3)

Reverse zipcode, add month and year, aggregate by ZIP, month, year

In [12]:
# temporarily drop columns not using (may use later)
sub_hosp_df = hosp_df_diag.drop(columns=['Unnamed: 0','ADM_TYPE','DIAG1','DIAG2','DIAG3'])

def reverse_zip(val):
    z = int(str(int(val))[::-1])
    return z

# get zipcode from reversed zipcodes
sub_hosp_df['zip'] = sub_hosp_df['zipcode_R'].apply(reverse_zip)

# drop zipcode_r
sub_hosp_df.drop(columns=['zipcode_R'],inplace=True)

In [13]:
def admin_yr(val):
    a = int(val[-4:])
    return a

def admin_month(val):
    a = val[2:-4]
    return a


# get admission year from admission date (because year is actually referring to discharge date)
sub_hosp_df['AYEAR'] = sub_hosp_df['ADATE'].apply(admin_yr)

sub_hosp_df['AMONTH'] = sub_hosp_df['ADATE'].apply(admin_month)

# drop columns not necessary for aggregating
# year - discharge date, not admission date year
sub_hosp_df.drop(columns=['QID','ADATE', 'YEAR', 'index'], inplace=True)

In [14]:
cols_to_agg2 = {'ICU_DAY': sum, 'CCI_DAY': sum, 'LOS':'mean',
       'Parkinson_pdx2dx_25':sum, 'Alzheimer_pdx2dx_25':sum, 'Dementia_pdx2dx_25':sum,
       'CHF_pdx2dx_25':sum, 'AMI_pdx2dx_25':sum, 'COPD_pdx2dx_25':sum, 'DM_pdx2dx_25':sum,
       'Stroke_pdx2dx_25':sum, 'CVD_pdx2dx_25':sum, 'CSD_pdx2dx_25':sum,
       'Ischemic_stroke_pdx2dx_25':sum, 'Hemo_Stroke_pdx2dx_25':sum,
       'neo_140_149':sum, 'neo_150_159':sum, 'neo_160_165':sum,
       'neo_170_176':sum, 'neo_179_189':sum, 'neo_190_199':sum, 'neo_200_209':sum,
       'neo_210_229':sum, 'neo_230_234':sum, 'neo_235_238':sum, 'neo_239':sum}

# groupby and aggregate by zipcode
sub_hosp_df3 = sub_hosp_df.groupby(['zip', 'AYEAR', 'AMONTH']).aggregate(cols_to_agg2)

# reset index
sub_hosp_df3.reset_index(inplace=True)

In [15]:
sub_hosp_df4 = sub_hosp_df3[sub_hosp_df3.AYEAR >=2011]
sub_hosp_df4.shape

(2170438, 29)

In [16]:
# save results
sub_hosp_df4.to_csv('hosp_aggregated_zip_2011_2016.csv')

# Appendix Code

Aggregate Medicare Monthly deaths

In [ ]:
c_size = 100000
reader = pd.read_csv('medicare_pop_2011_2016.csv', chunksize=c_size)
df_ls = []

for chunk in reader:
    # read in chunk at a time
    
    c = chunk.drop(columns=['Unnamed: 0'])
    if c.population.isna().sum() != c_size:
        c2 =c.dropna()
        df_ls.append(c2)
        
merged_df = pd.concat(df_ls)

agg_pop = merged_df.groupby(['zip','year']).mean()
agg_pop = agg_pop.reset_index()

agg_pop.to_csv('medicare_pop_aggregated_2011_2016.csv')

In [11]:
# appendix code
cols_to_agg = {'ADATE': min, 'ICU_DAY': 'mean', 'CCI_DAY': 'mean', 'YEAR':min, 'LOS':'mean',
       'Parkinson_pdx2dx_25':sum, 'Alzheimer_pdx2dx_25':sum, 'Dementia_pdx2dx_25':sum,
       'CHF_pdx2dx_25':sum, 'AMI_pdx2dx_25':sum, 'COPD_pdx2dx_25':sum, 'DM_pdx2dx_25':sum,
       'Stroke_pdx2dx_25':sum, 'CVD_pdx2dx_25':sum, 'CSD_pdx2dx_25':sum,
       'Ischemic_stroke_pdx2dx_25':sum, 'Hemo_Stroke_pdx2dx_25':sum, 'zip':min}

# groupby and aggregate by QID
# sub_hosp_df2 = sub_hosp_df.groupby('QID').aggregate(cols_to_agg)

def admin_yr(val):
    a = int(val[-4:])
    return a

def admin_month(val):
    a = val[:2]
    return a
    
def admin_day(val):
    a = val[2:-4]
    return a

# get admission year from admission date (because year is actually referring to discharge date)
# sub_hosp_df2['AYEAR'] = sub_hosp_df2['ADATE'].apply(admin_yr)

# get admission day of month
sub_hosp_df2['ADAY'] = sub_hosp_df2['ADATE'].apply(admin_day)

# get admission month
sub_hosp_df2['AMONTH'] = sub_hosp_df2['ADATE'].apply(admin_month)

# reset index
sub_hosp_df2.reset_index(inplace=True)

Census Data

In [4]:
census_zips = pd.read_csv('../../ci3_confounders/data_for_analysis/prepped_census/zcta_tract_rel_10.txt')

census_data = pd.read_csv('../../ci3_confounders/data_for_analysis/prepped_census/census_county_interpolated.csv')
# census_data2 = pd.read_csv('../../ci3_confounders/data_for_analysis/prepped_census/census_county_uninterpolated.csv')

In [18]:
census_zips.columns, census_data.columns


(Index(['ZCTA5', 'STATE', 'COUNTY', 'TRACT', 'GEOID', 'POPPT', 'HUPT', 'AREAPT',
        ' AREALANDPT', 'ZPOP', 'ZHU', 'ZAREA', 'ZAREALAND', 'TRPOP', 'TRHU',
        'TRAREA', 'TRAREALAND', 'ZPOPPCT', 'ZHUPCT', 'ZAREAPCT', 'ZAREALANDPCT',
        'TRPOPPCT', 'TRHUPCT', 'TRAREAPCT', 'TRAREALANDPCT'],
       dtype='object'),
 Index(['fips', 'NAME', 'land_area', 'year', 'hispanic_pct', 'poverty',
        'poverty_mcare', 'population', 'median_house_value', 'blk_pct',
        'white_pct', 'native_pct', 'asian_pct', 'no_grad', 'no_grad_mcare',
        'median_household_income', 'owner_occupied', 'median_age',
        'age_pct_0_14', 'age_pct_15_44', 'age_pct_45_65', 'age_pct_65_plus',
        'population_density'],
       dtype='object'))

In [11]:
census_data2 = census_data.merge(census_zips[['ZCTA5','COUNTY','STATE']], how='left', left_on=['fips'], right_on=['COUNTY'])

In [15]:
census_data2.ZCTA5.isna().sum(), census_data2.shape

(64660, (64660, 26))

In [18]:
# np.unique(census_data.year,return_counts=True)
sub_census = census_data[census_data['year']>=2011]
np.unique(sub_census.year,return_counts=True)

(array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]),
 array([3233, 3233, 3233, 3233, 3233, 3233, 3233, 3233]))

Environmental Data, Exposure

In [2]:
temp = pd.read_csv("../../ci3_confounders/data_for_analysis/earth_engine/temperature/temperature_daily_zipcode_combined.csv")

In [62]:
temp_seasonal= pd.read_csv("../../ci3_confounders/data_for_analysis/earth_engine/temperature/temperature_seasonal_zipcode_combined.csv")
temp_seasonal.dropna(inplace=True)

In [4]:
# subset to only include years after 2011
sub_temp = temp[temp.year >=2011]

def convert_datetime(val):
    ans = pd.to_datetime(val, format='%Y-%m-%d')
    return ans

sub_temp['date'] = sub_temp['date'].apply(convert_datetime)

<ipython-input-4-5a2523a01b0e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_temp['date'] = sub_temp['date'].apply(convert_datetime)


In [5]:
# get month, aggregate by year, month, zip
sub_temp['month'] = sub_temp['date'].dt.month

# drop columns not needed to aggregate
sub_temp.drop(columns=['date'], inplace=True)

# drop nan values
sub_temp.dropna(inplace=True)

<ipython-input-5-8e118fbca494>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_temp['month'] = sub_temp['date'].dt.month
/nfs/projects/n/nsaph_common/conda/envs/nsaph_tensorflow/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-5-8e118fbca494>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [54]:
# aggregate monthly
temp_cols_agg = {'tmmx':'mean', 'rmax':'mean', 'pr':'mean'}
temp_agg = sub_temp.groupby(['ZIP','year', 'month']).aggregate(temp_cols_agg)

temp_agg.reset_index(inplace=True)

# save results
temp_agg.to_csv('monthly_temp_by_zip.csv')

Now do the same with humidity, precipitation, pm, ozone, no2, etc.

In [73]:
wind = pd.read_csv("../../ci3_confounders/data_for_analysis/earth_engine/wind/wind_daily_zipcode_combined.csv")

In [77]:
wind.dropna(inplace=True)

In [78]:
wind_sub = wind[wind['year'] >= 2011]
wind_sub.shape

(89725136, 5)